# Switching (a Copy!) of My World to Creative Mode

I'm planning out a Survival build, and I'm really unsure about the scale and the palette. I could prototype it in a superflat world  (like I did with my chest monster), but I really want to make sure that it works well in its surroundings. So I've made a copy, made a backup, deleted all the chunks that are outside of my immediate surroundings (because the save was getting massive!) and now I just have to swap the game mode in the NBT.

(yes, I know the LAN trick, but I want to do it permanently)

## Imports, Setup and Macros

In [1]:
import json
from os import environ
from pathlib import Path
from typing import Any, Collection, Dict, Set

import mutf8
import pandas as pd
from IPython.display import Markdown, display
from nbt import nbt, region

In [4]:
def format_file_size(path: Path) -> str:
    """Print the size of the specified file in
    human-readible form (KB / MB / GB)

    Parameters
    ----------
    path : Path
        The path to the file

    Returns
    -------
    str
        A prettily formatted file size

    Notes
    -----
    I would be shocked if there isn't a utility already built
    into the standard library to do this, but all I could find
    via Googling was a bunch of recipes and examples
    """
    size: float = path.stat().st_size  # in bytes
    for unit in ("B", "KB", "MB", "GB"):
        if size < 1024 / 2:
            return f"{size:.1f} {unit}"
        size = size / 1024
    return f"{size} TB"

In [2]:
def summarize_keystore(keystore: Dict[str, Any]) -> None:
    """Display a summary of the contents of a key-value store

    Parameters
    ----------
    keystore : dict
        The keystore to summarize

    Returns
    -------
    None
    """

    def _summarize_keystore(keystore: Dict[str, Any]) -> str:
        summary = ""
        for k, v in keystore.items():
            summary += f"\n - `{k}` : "
            if isinstance(v, (str, nbt.TAG_String)):
                summary += f'`"{v}"`'
            elif not isinstance(v, Collection):
                summary += f"`{str(v)}`"
            else:
                length = len(v)
                if 0 < length < 3:
                    summary += "\n"
                    if not isinstance(v, Dict):
                        v = {i: item for i, item in enumerate(v)}
                    summary += "\n".join(
                        (f"\t{line}" for line in _summarize_keystore(v).split("\n"))
                    )
                else:
                    summary += f"({len(v)} items)"
        return summary

    display(Markdown(_summarize_keystore(keystore)))

In [5]:
save_folder = Path(environ["SAVE_PATH"])

# make sure this is set correctly
for path in sorted(save_folder.glob("*")):
    print(f"- {path.name} ({'folder' if path.is_dir() else format_file_size(path)})")

- advancements (folder)
- data (folder)
- datapacks (folder)
- entities (folder)
- level.dat (12.6 KB)
- level.dat_old (12.6 KB)
- level11326231134829879582.dat (0.0 B)
- level14927678268100607923.dat (0.0 B)
- level1786655981796876926.dat (0.0 B)
- level4463453738642305340.dat (0.0 B)
- level6425531070021529407.dat (0.0 B)
- level7443636089696258371.dat (0.0 B)
- level8832581565660323154.dat (0.0 B)
- playerdata (folder)
- poi (folder)
- region (folder)
- serverconfig (folder)
- session.lock (8.0 B)
- stats (folder)


## Load up the ole `level.dat`

In [6]:
%%time
level = nbt.NBTFile(save_folder / "level.dat")

CPU times: user 11.4 ms, sys: 0 ns, total: 11.4 ms
Wall time: 10.5 ms


In [7]:
summarize_keystore(level["Data"])


 - `WanderingTraderSpawnChance` : `75`
 - `BorderCenterZ` : `0.0`
 - `Difficulty` : `3`
 - `BorderSizeLerpTime` : `0`
 - `raining` : `0`
 - `Time` : `29804685`
 - `GameType` : `0`
 - `ServerBrands` : 
	
	 - `0` : `"forge"`
	 - `1` : `"fabric"`
 - `BorderCenterX` : `0.0`
 - `BorderDamagePerBlock` : `0.2`
 - `BorderWarningBlocks` : `5.0`
 - `WorldGenSettings` : (4 items)
 - `DragonFight` : (6 items)
 - `BorderSizeLerpTarget` : `59999968.0`
 - `Version` : (4 items)
 - `DayTime` : `39456864`
 - `initialized` : `1`
 - `WasModded` : `1`
 - `allowCommands` : `0`
 - `WanderingTraderSpawnDelay` : `16800`
 - `CustomBossEvents` : (0 items)
 - `GameRules` : (35 items)
 - `Player` : (44 items)
 - `SpawnY` : `68`
 - `rainTime` : `67187`
 - `thunderTime` : `107506`
 - `SpawnZ` : `144`
 - `hardcore` : `0`
 - `WanderingTraderId` : (4 items)
 - `DifficultyLocked` : `0`
 - `SpawnX` : `-592`
 - `clearWeatherTime` : `0`
 - `thundering` : `0`
 - `SpawnAngle` : `0.0`
 - `version` : `19133`
 - `BorderSafeZone` : `5.0`
 - `LastPlayed` : `1651020439121`
 - `BorderWarningTime` : `15.0`
 - `ScheduledEvents` : (14 items)
 - `LevelName` : `"Esha Ness"`
 - `BorderSize` : `59999968.0`
 - `DataVersion` : `2865`
 - `DataPacks` : 
	
	 - `0` : `"Enabled"`
	 - `1` : `"Disabled"`

First thing is I need to change the level name

In [8]:
type(level["Data"]["LevelName"])

nbt.nbt.TAG_String

In [10]:
level["Data"]["LevelName"].value = "Esha Ness (Creative)"

Now, first thing that occurred to me was that I just needed to change `GameType` but [the wiki](https://minecraft.fandom.com/wiki/Java_Edition_level_format) says this field controls:

> The default game mode for the singleplayer player when they initially spawn. 0 is Survival, 1 is Creative, 2 is Adventure, 3 is Spectator. **Note: Singleplayer worlds do not use this field to save which game mode the player is currently in.**

That being said, `allowCommands` is exactly what I think it is:

> 1 or 0 (true/false) - true if cheats are enabled.

So I could just swap that to a 1 and use a command to change my game mode. Which I will do:

In [11]:
type(level["Data"]["allowCommands"])

nbt.nbt.TAG_Byte

In [12]:
type(level["Data"]["allowCommands"].value)

int

In [14]:
level["Data"]["allowCommands"].value = 1

But I still want to have a peek at `GameRules`

In [15]:
summarize_keystore(level["Data"]["GameRules"])


 - `doFireTick` : `"true"`
 - `maxCommandChainLength` : `"65536"`
 - `fireDamage` : `"true"`
 - `reducedDebugInfo` : `"false"`
 - `disableElytraMovementCheck` : `"false"`
 - `announceAdvancements` : `"true"`
 - `drowningDamage` : `"true"`
 - `forgiveDeadPlayers` : `"true"`
 - `commandBlockOutput` : `"true"`
 - `maxEntityCramming` : `"24"`
 - `doMobSpawning` : `"true"`
 - `disableRaids` : `"false"`
 - `doWeatherCycle` : `"true"`
 - `doDaylightCycle` : `"true"`
 - `showDeathMessages` : `"true"`
 - `doTileDrops` : `"true"`
 - `universalAnger` : `"false"`
 - `playersSleepingPercentage` : `"1"`
 - `doInsomnia` : `"true"`
 - `doImmediateRespawn` : `"false"`
 - `naturalRegeneration` : `"true"`
 - `doMobLoot` : `"true"`
 - `fallDamage` : `"true"`
 - `keepInventory` : `"false"`
 - `doEntityDrops` : `"true"`
 - `doLimitedCrafting` : `"false"`
 - `mobGriefing` : `"true"`
 - `randomTickSpeed` : `"3"`
 - `spawnRadius` : `"10"`
 - `doTraderSpawning` : `"true"`
 - `freezeDamage` : `"true"`
 - `logAdminCommands` : `"true"`
 - `spectatorsGenerateChunks` : `"true"`
 - `sendCommandFeedback` : `"true"`
 - `doPatrolSpawning` : `"true"`

It's not there! But wait--it wouldn't be, would it. It would be under...

In [16]:
summarize_keystore(level["Data"]["Player"])


 - `Brain` : 
	
	 - `0` : `"memories"`
 - `HurtByTimestamp` : `10228`
 - `SleepTimer` : `0`
 - `SpawnForced` : `0`
 - `Attributes` : (6 items)
 - `Invulnerable` : `0`
 - `FallFlying` : `0`
 - `PortalCooldown` : `0`
 - `AbsorptionAmount` : `0.0`
 - `abilities` : (7 items)
 - `FallDistance` : `0.0`
 - `recipeBook` : (10 items)
 - `DeathTime` : `0`
 - `XpSeed` : `-1962982261`
 - `XpTotal` : `5199`
 - `UUID` : (4 items)
 - `playerGameType` : `0`
 - `SpawnDimension` : `"minecraft:overworld"`
 - `Tags` : 
	
	 - `0` : `"ch_east"`
 - `seenCredits` : `1`
 - `Motion` : (3 items)
 - `SpawnY` : `112`
 - `Health` : `20.0`
 - `SpawnZ` : `-3929`
 - `foodSaturationLevel` : `9.40000057220459`
 - `SpawnX` : `-1510`
 - `Air` : `300`
 - `OnGround` : `1`
 - `Dimension` : `"minecraft:overworld"`
 - `SpawnAngle` : `-144.38970947265625`
 - `Rotation` : 
	
	 - `0` : `-71.8480224609375`
	 - `1` : `30.45000457763672`
 - `XpLevel` : `44`
 - `Score` : `5199`
 - `Pos` : (3 items)
 - `Fire` : `-20`
 - `XpP` : `0.13543184101581573`
 - `EnderItems` : (18 items)
 - `DataVersion` : `2865`
 - `foodLevel` : `20`
 - `foodExhaustionLevel` : `1.108359456062317`
 - `HurtTime` : `0`
 - `SelectedItemSlot` : `1`
 - `Inventory` : (16 items)
 - `foodTickTimer` : `0`

Bingo

In [17]:
level["Data"]["Player"]["playerGameType"].value = 1

## That was Easy

In [18]:
level.write_file(save_folder / "level.dat")